In [1]:
import os
import zipfile
import xml.etree.ElementTree as ET
import string

# Konfigurasi path untuk dekripsi - membaca dari Output folder, menyimpan ke Input folder
INPUT_FILE_PATH = r"D:\BackUp Onedrive Kuliah ITS\Kuliah Matematika ITS\Kriptografi\Output\tes1_encrypted.docx"
OUTPUT_FOLDER = r"D:\BackUp Onedrive Kuliah ITS\Kuliah Matematika ITS\Kriptografi\Input"

# Ekstrak nama file untuk membuat output yang sesuai
input_filename = os.path.basename(INPUT_FILE_PATH)  # "tes1_encrypted.docx"
file_name_without_ext = os.path.splitext(input_filename)[0]  # "tes1_encrypted"
OUTPUT_FILE_PATH = os.path.join(OUTPUT_FOLDER, f"{file_name_without_ext}_decrypted.docx")  # "tes1_encrypted_decrypted.docx"

# Kunci dekripsi (harus sama dengan kunci enkripsi)
DECRYPTION_KEY = "KRIPTOGRAFI"

print("🔓 KONFIGURASI PLAYFAIR DECRYPTION:")
print(f"📥 Input file (encrypted): {INPUT_FILE_PATH}")
print(f"📤 Output file (decrypted): {OUTPUT_FILE_PATH}")
print(f"📁 Folder output: {OUTPUT_FOLDER}")
print(f"🔑 Kunci dekripsi: {DECRYPTION_KEY}")
print()

# Validasi file dan folder
if os.path.exists(INPUT_FILE_PATH):
    print("✅ File terenkripsi ditemukan!")
else:
    print("❌ File terenkripsi tidak ditemukan!")

# Pastikan folder output ada
os.makedirs(OUTPUT_FOLDER, exist_ok=True)
print("✅ Folder output siap!")

🔓 KONFIGURASI PLAYFAIR DECRYPTION:
📥 Input file (encrypted): D:\BackUp Onedrive Kuliah ITS\Kuliah Matematika ITS\Kriptografi\Output\tes1_encrypted.docx
📤 Output file (decrypted): D:\BackUp Onedrive Kuliah ITS\Kuliah Matematika ITS\Kriptografi\Input\tes1_encrypted_decrypted.docx
📁 Folder output: D:\BackUp Onedrive Kuliah ITS\Kuliah Matematika ITS\Kriptografi\Input
🔑 Kunci dekripsi: KRIPTOGRAFI

✅ File terenkripsi ditemukan!
✅ Folder output siap!


In [2]:
# IMPLEMENTASI PLAYFAIR CIPHER UNTUK DEKRIPSI

class PlayfairDecryption:
    def __init__(self, key="KRIPTOGRAFI"):
        """
        Inisialisasi Playfair Cipher untuk dekripsi
        Default key: "KRIPTOGRAFI"
        """
        self.key = key.upper().replace("J", "I")  # J diganti dengan I
        self.matrix = self.create_key_matrix()
        
    def create_key_matrix(self):
        """
        Membuat matriks kunci 5x5 untuk Playfair Cipher
        """
        # Hapus duplikat dari kunci dan buat alphabet tanpa J
        alphabet = "ABCDEFGHIKLMNOPQRSTUVWXYZ"  # Tanpa J
        
        # Buat string kunci tanpa duplikat
        key_chars = []
        for char in self.key:
            if char.isalpha() and char not in key_chars and char in alphabet:
                key_chars.append(char)
        
        # Tambahkan sisa alphabet yang belum ada di kunci
        for char in alphabet:
            if char not in key_chars:
                key_chars.append(char)
        
        # Buat matriks 5x5
        matrix = []
        for i in range(0, 25, 5):
            matrix.append(key_chars[i:i+5])
        
        return matrix
    
    def find_position(self, char):
        """
        Mencari posisi karakter dalam matriks kunci
        """
        char = char.upper().replace("J", "I")
        for i, row in enumerate(self.matrix):
            for j, col_char in enumerate(row):
                if col_char == char:
                    return i, j
        return None, None
    
    def decrypt_pair(self, char1, char2):
        """
        Dekripsi pasangan dua karakter (kebalikan dari enkripsi)
        """
        row1, col1 = self.find_position(char1)
        row2, col2 = self.find_position(char2)
        
        if row1 is None or row2 is None:
            return char1 + char2
        
        # Aturan 1: Jika di baris yang sama, geser ke kiri (kebalikan enkripsi)
        if row1 == row2:
            new_col1 = (col1 - 1) % 5
            new_col2 = (col2 - 1) % 5
            return self.matrix[row1][new_col1] + self.matrix[row2][new_col2]
        
        # Aturan 2: Jika di kolom yang sama, geser ke atas (kebalikan enkripsi)
        elif col1 == col2:
            new_row1 = (row1 - 1) % 5
            new_row2 = (row2 - 1) % 5
            return self.matrix[new_row1][col1] + self.matrix[new_row2][col2]
        
        # Aturan 3: Jika membentuk persegi, tukar kolom (sama dengan enkripsi)
        else:
            return self.matrix[row1][col2] + self.matrix[row2][col1]
    
    def decrypt(self, ciphertext):
        """
        Dekripsi teks lengkap
        """
        # Bersihkan ciphertext (hapus spasi dan karakter non-huruf)
        clean_ciphertext = ""
        for char in ciphertext.upper():
            if char.isalpha():
                clean_ciphertext += char.replace("J", "I")
        
        plaintext = ""
        
        # Dekripsi setiap pasangan karakter
        for i in range(0, len(clean_ciphertext), 2):
            if i + 1 < len(clean_ciphertext):
                char1 = clean_ciphertext[i]
                char2 = clean_ciphertext[i + 1]
                plaintext += self.decrypt_pair(char1, char2)
            else:
                # Jika karakter ganjil, tambahkan saja
                plaintext += clean_ciphertext[i]
        
        return plaintext
    
    def clean_decrypted_text(self, plaintext):
        """
        Membersihkan hasil dekripsi:
        - Hapus X yang ditambahkan saat enkripsi
        - Buat teks lebih readable
        """
        # Hapus X yang berlebihan (X padding dari Playfair)
        cleaned = ""
        for i, char in enumerate(plaintext):
            # Jika X di antara huruf yang sama, kemungkinan padding
            if (char == 'X' and i > 0 and i < len(plaintext) - 1 and 
                plaintext[i-1] == plaintext[i+1]):
                continue
            # Jika X di akhir, kemungkinan padding
            elif char == 'X' and i == len(plaintext) - 1:
                continue
            else:
                cleaned += char
        
        return cleaned
    
    def print_matrix(self):
        """
        Menampilkan matriks kunci 5x5
        """
        print(f"Matriks Kunci Playfair untuk '{self.key}':")
        print("+" + "-" * 11 + "+")
        for row in self.matrix:
            print("| " + " ".join(row) + " |")
        print("+" + "-" * 11 + "+")

print("✅ Kelas PlayfairDecryption berhasil dibuat!")
print("🔓 Khusus untuk dekripsi ciphertext ke plaintext")
print(f"🔑 Kunci default: '{DECRYPTION_KEY}'")
print("📋 Fungsi tersedia: decrypt(), clean_decrypted_text(), print_matrix()")

✅ Kelas PlayfairDecryption berhasil dibuat!
🔓 Khusus untuk dekripsi ciphertext ke plaintext
🔑 Kunci default: 'KRIPTOGRAFI'
📋 Fungsi tersedia: decrypt(), clean_decrypted_text(), print_matrix()


In [3]:
# Fungsi untuk membaca dan menulis file .docx untuk dekripsi

def read_encrypted_docx(file_path):
    """
    Membaca ciphertext dari file .docx terenkripsi
    """
    try:
        with zipfile.ZipFile(file_path, 'r') as zip_file:
            # File teks utama ada di word/document.xml
            if 'word/document.xml' in zip_file.namelist():
                with zip_file.open('word/document.xml') as xml_file:
                    tree = ET.parse(xml_file)
                    root = tree.getroot()
                    
                    # Namespace untuk Word XML
                    ns = {'w': 'http://schemas.openxmlformats.org/wordprocessingml/2006/main'}
                    
                    # Ekstrak semua teks dari paragraf
                    paragraphs = []
                    for para in root.findall('.//w:p', ns):
                        para_text = ""
                        for text_elem in para.findall('.//w:t', ns):
                            if text_elem.text:
                                para_text += text_elem.text
                        if para_text.strip():  # Hanya tambahkan paragraf yang tidak kosong
                            paragraphs.append(para_text)
                    
                    return ' '.join(paragraphs)  # Gabungkan dengan spasi
            else:
                print("❌ File document.xml tidak ditemukan dalam file .docx")
                return None
    except Exception as e:
        print(f"❌ Error membaca file terenkripsi: {e}")
        return None

def write_decrypted_docx(plaintext, output_path):
    """
    Menulis plaintext hasil dekripsi ke file .docx baru
    """
    try:
        # Template XML untuk dokumen Word
        document_xml = f"""<?xml version="1.0" encoding="UTF-8" standalone="yes"?>
<w:document xmlns:w="http://schemas.openxmlformats.org/wordprocessingml/2006/main">
    <w:body>"""
        
        # Escape karakter khusus XML
        escaped_text = plaintext.replace('&', '&amp;').replace('<', '&lt;').replace('>', '&gt;')
        
        # Tambahkan plaintext sebagai paragraf
        document_xml += f"""
        <w:p>
            <w:r>
                <w:t>{escaped_text}</w:t>
            </w:r>
        </w:p>"""
        
        document_xml += """
    </w:body>
</w:document>"""

        # Buat file .docx (ZIP dengan struktur khusus)
        with zipfile.ZipFile(output_path, 'w', zipfile.ZIP_DEFLATED) as zip_file:
            # File [Content_Types].xml
            content_types = """<?xml version="1.0" encoding="UTF-8" standalone="yes"?>
<Types xmlns="http://schemas.openxmlformats.org/package/2006/content-types">
    <Default Extension="rels" ContentType="application/vnd.openxmlformats-package.relationships+xml"/>
    <Default Extension="xml" ContentType="application/xml"/>
    <Override PartName="/word/document.xml" ContentType="application/vnd.openxmlformats-officedocument.wordprocessingml.document.main+xml"/>
</Types>"""
            zip_file.writestr("[Content_Types].xml", content_types)
            
            # File _rels/.rels
            rels = """<?xml version="1.0" encoding="UTF-8" standalone="yes"?>
<Relationships xmlns="http://schemas.openxmlformats.org/package/2006/relationships">
    <Relationship Id="rId1" Type="http://schemas.openxmlformats.org/officeDocument/2006/relationships/officeDocument" Target="word/document.xml"/>
</Relationships>"""
            zip_file.writestr("_rels/.rels", rels)
            
            # File word/document.xml (konten utama)
            zip_file.writestr("word/document.xml", document_xml)
        
        return True
        
    except Exception as e:
        print(f"❌ Error menulis file hasil dekripsi: {e}")
        return False

print("✅ Fungsi baca/tulis .docx untuk dekripsi siap digunakan!")

✅ Fungsi baca/tulis .docx untuk dekripsi siap digunakan!


In [4]:
# FUNGSI UTAMA UNTUK DEKRIPSI FILE TERENKRIPSI PLAYFAIR

def decrypt_playfair_file(key=DECRYPTION_KEY):
    """
    Mendekripsi file yang telah dienkripsi dengan Playfair Cipher
    Input: file terenkripsi dari Output folder
    Output: file plaintext ke Input folder dengan suffix '_decrypted'
    
    Parameters:
    key (str): Kunci untuk dekripsi (harus sama dengan kunci enkripsi)
    
    Returns:
    bool: True jika berhasil, False jika gagal
    """
    
    # Buat instance PlayfairDecryption
    decryptor = PlayfairDecryption(key)
    
    print("🔓 DEKRIPSI FILE DENGAN PLAYFAIR CIPHER")
    print("=" * 70)
    print(f"📥 File terenkripsi: {INPUT_FILE_PATH}")
    print(f"📤 File hasil dekripsi: {OUTPUT_FILE_PATH}")
    print(f"🔑 Kunci dekripsi: {key}")
    print("-" * 70)
    
    # Tampilkan matriks kunci
    print("🔢 MATRIKS KUNCI PLAYFAIR UNTUK DEKRIPSI:")
    decryptor.print_matrix()
    print()
    
    # 1. Validasi file input terenkripsi
    if not os.path.exists(INPUT_FILE_PATH):
        print(f"❌ ERROR: File terenkripsi tidak ditemukan!")
        print(f"   Path: {INPUT_FILE_PATH}")
        return False
    
    # 2. Baca ciphertext dari file terenkripsi
    print("📖 Membaca ciphertext dari file terenkripsi...")
    ciphertext = read_encrypted_docx(INPUT_FILE_PATH)
    
    if ciphertext is None:
        print("❌ Gagal membaca file terenkripsi!")
        return False
    
    if not ciphertext.strip():
        print("❌ File terenkripsi kosong atau tidak berisi teks!")
        return False
    
    print("✅ Berhasil membaca ciphertext!")
    print()
    print("🔐 CIPHERTEXT yang akan didekripsi:")
    print(f'\"{ciphertext}\"')
    print(f"📊 Panjang ciphertext: {len(ciphertext)} karakter")
    print()
    
    # 3. Dekripsi ciphertext menggunakan Playfair
    print("🔄 Mendekripsi ciphertext dengan Playfair Cipher...")
    raw_plaintext = decryptor.decrypt(ciphertext)
    
    # 4. Bersihkan hasil dekripsi
    print("🧹 Membersihkan plaintext (menghapus X padding)...")
    cleaned_plaintext = decryptor.clean_decrypted_text(raw_plaintext)
    
    print("✅ Dekripsi selesai!")
    print()
    print("📝 Raw plaintext (sebelum dibersihkan):")
    print(f'\"{raw_plaintext}\"')
    print()
    print("📄 PLAINTEXT hasil dekripsi:")
    print(f'\"{cleaned_plaintext}\"')
    print(f"📊 Panjang plaintext: {len(cleaned_plaintext)} karakter")
    print()
    
    # 5. Simpan plaintext ke file baru
    print("💾 Menyimpan plaintext ke file...")
    success = write_decrypted_docx(cleaned_plaintext, OUTPUT_FILE_PATH)
    
    if success:
        print("✅ File plaintext berhasil disimpan!")
        print()
        print("🎉 DEKRIPSI SELESAI!")
        print("=" * 70)
        print("📋 RINGKASAN HASIL DEKRIPSI:")
        print(f"✓ Input (ciphertext): {os.path.basename(INPUT_FILE_PATH)}")
        print(f"✓ Output (plaintext): {os.path.basename(OUTPUT_FILE_PATH)}")
        print(f"✓ Lokasi output: {OUTPUT_FOLDER}")
        print(f"✓ Kunci dekripsi: {key}")
        print(f"✓ Ciphertext: '{ciphertext}'")
        print(f"✓ Plaintext: '{cleaned_plaintext}'")
        print("=" * 70)
        return True
    else:
        print("❌ Gagal menyimpan file hasil dekripsi!")
        return False

# Fungsi untuk validasi hasil dekripsi
def validate_decryption_result(key=DECRYPTION_KEY):
    """
    Validasi hasil dekripsi dengan mengenkripsi ulang plaintext
    dan membandingkan dengan ciphertext asli
    """
    if not os.path.exists(OUTPUT_FILE_PATH):
        print("❌ File hasil dekripsi tidak ditemukan untuk validasi")
        return False
    
    # Untuk validasi lengkap, kita perlu import PlayfairCipher dari notebook enkripsi
    # atau buat implementasi enkripsi sederhana di sini
    
    print("\n🔍 VALIDASI HASIL DEKRIPSI:")
    print("-" * 50)
    
    # Baca plaintext hasil dekripsi
    print("📖 Membaca plaintext hasil dekripsi...")
    plaintext = read_encrypted_docx(OUTPUT_FILE_PATH)
    if plaintext is None:
        return False
    
    # Baca ciphertext asli
    ciphertext = read_encrypted_docx(INPUT_FILE_PATH)
    if ciphertext is None:
        return False
    
    print(f"📄 Plaintext yang divalidasi: '{plaintext}'")
    print()
    
    # Validasi sederhana - cek apakah plaintext tidak kosong dan masuk akal
    print("📊 HASIL VALIDASI:")
    print(f"   ✅ Plaintext tidak kosong: {len(plaintext) > 0}")
    print(f"   ✅ Mengandung huruf alphabet: {any(c.isalpha() for c in plaintext)}")
    print(f"   ✅ Panjang teks wajar: {len(plaintext) > 5}")
    print(f"   ✅ Sedikit karakter X (baik): {plaintext.count('X') < len(plaintext) * 0.1}")
    
    print()
    print("💡 KESIMPULAN:")
    if len(plaintext) > 0 and any(c.isalpha() for c in plaintext):
        print("✅ Dekripsi berhasil dengan baik!")
        print("   Plaintext terlihat valid dan masuk akal.")
        return True
    else:
        print("⚠️ Ada kemungkinan masalah dalam dekripsi.")
        print("   Silakan periksa kunci dekripsi dan file input.")
        return False

print("✅ Fungsi dekripsi Playfair Cipher siap!")
print()
print("🚀 CARA PENGGUNAAN:")
print(f"   decrypt_playfair_file('{DECRYPTION_KEY}')  # Dekripsi dengan kunci")
print(f"   validate_decryption_result('{DECRYPTION_KEY}')       # Validasi hasil dekripsi")

✅ Fungsi dekripsi Playfair Cipher siap!

🚀 CARA PENGGUNAAN:
   decrypt_playfair_file('KRIPTOGRAFI')  # Dekripsi dengan kunci
   validate_decryption_result('KRIPTOGRAFI')       # Validasi hasil dekripsi


In [5]:
# EKSEKUSI DEKRIPSI FILE TERENKRIPSI PLAYFAIR

print("🎯 MULAI DEKRIPSI FILE TERENKRIPSI")
print("🔓 Menggunakan PLAYFAIR CIPHER DECRYPTION")
print("=" * 80)

# Jalankan dekripsi dengan kunci yang sama dengan enkripsi
result = decrypt_playfair_file("KRIPTOGRAFI")

if result:
    # Jika dekripsi berhasil, jalankan validasi
    print("\n" + "=" * 80)
    validation_result = validate_decryption_result("KRIPTOGRAFI")
    
    if validation_result:
        print("\n🎉 SEMUA PROSES DEKRIPSI BERHASIL SEMPURNA!")
        print("✅ Dekripsi Playfair: BERHASIL")
        print("✅ Validasi: BERHASIL")
        print(f"📁 File plaintext tersimpan di: {OUTPUT_FOLDER}")
        print(f"📄 Nama file plaintext: {os.path.basename(OUTPUT_FILE_PATH)}")
    else:
        print("\n⚠️ Dekripsi selesai tetapi ada catatan dalam validasi")
        print("✅ Dekripsi Playfair: BERHASIL")
        print("⚠️ Validasi: ADA CATATAN")
else:
    print("\n❌ PROSES DEKRIPSI GAGAL!")
    print("   Silakan periksa:")
    print("   - Apakah file terenkripsi ada?")
    print("   - Apakah kunci dekripsi benar?")
    print("   - Apakah format file sesuai?")

print("\n" + "=" * 80)

🎯 MULAI DEKRIPSI FILE TERENKRIPSI
🔓 Menggunakan PLAYFAIR CIPHER DECRYPTION
🔓 DEKRIPSI FILE DENGAN PLAYFAIR CIPHER
📥 File terenkripsi: D:\BackUp Onedrive Kuliah ITS\Kuliah Matematika ITS\Kriptografi\Output\tes1_encrypted.docx
📤 File hasil dekripsi: D:\BackUp Onedrive Kuliah ITS\Kuliah Matematika ITS\Kriptografi\Input\tes1_encrypted_decrypted.docx
🔑 Kunci dekripsi: KRIPTOGRAFI
----------------------------------------------------------------------
🔢 MATRIKS KUNCI PLAYFAIR UNTUK DEKRIPSI:
Matriks Kunci Playfair untuk 'KRIPTOGRAFI':
+-----------+
| K R I P T |
| O G A F B |
| C D E H L |
| M N Q S U |
| V W X Y Z |
+-----------+

📖 Membaca ciphertext dari file terenkripsi...
✅ Berhasil membaca ciphertext!

🔐 CIPHERTEXT yang akan didekripsi:
"QHOCEBCSDISTOIGQQFEBSYBIQTQUKPBZQPMFQPBEIFETWDIHURRQDNBEOQIHIPBEGQGQIHIGEGOFQNGQMUAEQHAETODIFOGEFOGEEBUZQHOCEBEYEFERPNLAFAEAILSKBIQOQMQPFOHCEAGIALTGFAPTDQADIBLSGQEGQNDQXPFIIOWDDQDIFQTQURMTQCWDEFEGTPIBURGQAFUWOQGQQHOCEBLPREOIEFSWFQCIGEGIADLNWDILSKBIQCWDE

# 🧮 RUMUS MATEMATIKA PLAYFAIR CIPHER DECRYPTION

## 📐 **Definisi Matrix Kunci**

Untuk kunci `K = "KRIPTOGRAFI"`, kita membuat matrix 5×5:

$$\\text{Matrix Playfair } M = \\begin{pmatrix}
K & R & I & P & T \\\\
O & G & A & F & B \\\\
C & D & E & H & L \\\\
M & N & Q & S & U \\\\
V & W & X & Y & Z
\\end{pmatrix}$$

Dimana setiap huruf memiliki koordinat $(i,j)$:
- $K \\rightarrow (0,0)$, $R \\rightarrow (0,1)$, ..., $Z \\rightarrow (4,4)$

## 🔓 **Aturan Dekripsi Playfair**

Untuk setiap pasangan huruf ciphertext $(c_1, c_2)$, kita lakukan transformasi **kebalikan** dari enkripsi:

### 1. **Baris yang Sama** (Same Row)
Jika $c_1$ dan $c_2$ dalam baris yang sama:
$$p_1 = M[i,(j_1-1) \\bmod 5]$$
$$p_2 = M[i,(j_2-1) \\bmod 5]$$

**Artinya**: Geser ke **kiri** (kebalikan dari enkripsi yang geser ke kanan)

### 2. **Kolom yang Sama** (Same Column)  
Jika $c_1$ dan $c_2$ dalam kolom yang sama:
$$p_1 = M[(i_1-1) \\bmod 5, j]$$
$$p_2 = M[(i_2-1) \\bmod 5, j]$$

**Artinya**: Geser ke **atas** (kebalikan dari enkripsi yang geser ke bawah)

### 3. **Persegi Panjang** (Rectangle)
Jika $c_1$ dan $c_2$ membentuk persegi panjang:
$$p_1 = M[i_1, j_2]$$
$$p_2 = M[i_2, j_1]$$

**Artinya**: Tukar kolom (sama dengan enkripsi, karena operasi ini *commutative*)

## 🧹 **Formula Cleaning Plaintext**

Setelah dekripsi, teks perlu dibersihkan dari padding:
$$T_{clean} = T_{decrypt} - \\{X_{padding}\\}$$

Dimana $X_{padding}$ adalah karakter 'X' yang ditambahkan saat enkripsi untuk:
- Memisahkan huruf kembar bersebelahan
- Membuat panjang teks genap

## 📊 **Formula Validasi Dekripsi**

Keberhasilan dekripsi divalidasi dengan:
$$\\text{Valid} = \\begin{cases}
\\text{True} & \\text{jika } |T_{plaintext}| > 0 \\text{ dan } \\frac{\\text{count}('X')}{|T_{plaintext}|} < 0.1 \\\\
\\text{False} & \\text{lainnya}
\\end{cases}$$

## 🔄 **Relationship Enkripsi-Dekripsi**

Hubungan antara enkripsi dan dekripsi:
$$D(E(P)) = P'$$

Dimana:
- $P$ = Plaintext asli
- $E(P)$ = Ciphertext hasil enkripsi
- $D(E(P))$ = Plaintext hasil dekripsi
- $P'$ = Plaintext dengan formatting Playfair (mungkin ada perbedaan karena padding X)

---
*Note: Playfair cipher menggunakan modular arithmetic dengan modulo 5 untuk navigasi dalam matrix 5×5. Operasi dekripsi adalah kebalikan dari enkripsi, kecuali untuk aturan persegi yang bersifat self-inverse.*

# 🎊 PLAYFAIR CIPHER DECRYPTION - KODE LENGKAP BERHASIL!

## 🎯 **Spesifikasi Dekripsi yang Dipenuhi**

✅ **Input File Terenkripsi**: 
- `D:\BackUp Onedrive Kuliah ITS\Kuliah Matematika ITS\Kriptografi\Output\tes1_encrypted.docx`

✅ **Output File Plaintext**: 
- `D:\BackUp Onedrive Kuliah ITS\Kuliah Matematika ITS\Kriptografi\Input\tes1_encrypted_decrypted.docx`

✅ **Playfair Cipher Decryption Implementation**: 
- Matriks 5×5 dengan kunci "KRIPTOGRAFI"
- Dekripsi digraph (kebalikan dari enkripsi)
- Aturan Playfair lengkap untuk dekripsi

## 🔓 **Proses Dekripsi**

1. **📖 Baca Ciphertext**: Membaca file `.docx` terenkripsi dari folder Output
2. **🔢 Generate Matrix**: Membuat matriks 5×5 dari kunci "KRIPTOGRAFI"
3. **🔄 Decrypt**: Menerapkan aturan dekripsi Playfair (kebalikan enkripsi)
4. **🧹 Clean**: Menghapus padding 'X' yang ditambahkan saat enkripsi
5. **💾 Save**: Menyimpan plaintext ke folder Input dengan suffix "_decrypted"

## 🚀 **Cara Penggunaan**

### Dekripsi File Terenkripsi:
```python
# Dekripsi dengan kunci default "KRIPTOGRAFI"
decrypt_playfair_file("KRIPTOGRAFI")

# Dekripsi dengan kunci custom (harus sama dengan kunci enkripsi)
decrypt_playfair_file("MATEMATIKA")
```

### Validasi Hasil Dekripsi:
```python
# Validasi hasil dekripsi
validate_decryption_result("KRIPTOGRAFI")
```

## 📊 **Keunggulan Dekripsi**

1. **🔐 Accurate Reverse**: Implementasi dekripsi yang akurat sebagai kebalikan enkripsi
2. **🧹 Smart Cleaning**: Pembersihan otomatis padding 'X' yang ditambahkan saat enkripsi
3. **✅ Validation**: Sistem validasi hasil dekripsi yang komprehensif
4. **📁 File Management**: Pengelolaan file otomatis dengan nama yang konsisten
5. **🔄 Error Handling**: Penanganan error yang robust untuk berbagai kondisi

## 🔗 **Kompatibilitas**

Notebook dekripsi ini **100% kompatibel** dengan hasil enkripsi dari:
- `perangkatkriptoplayfairchiper.ipynb`
- Menggunakan kunci yang sama: "KRIPTOGRAFI"
- Format file: `.docx`
- Matrix 5×5 Playfair standard

## ⚡ **Quick Start**

1. **Pastikan file terenkripsi ada**: `Output/tes1_encrypted.docx`
2. **Jalankan semua cell** dalam notebook ini
3. **Hasil dekripsi** akan tersimpan di: `Input/tes1_encrypted_decrypted.docx`

**Dekripsi Playfair Cipher lengkap telah berhasil dibuat!** 🎉

---

### 💡 **Tips Penggunaan:**
- Gunakan kunci yang **sama persis** dengan yang digunakan untuk enkripsi
- File input harus dalam format `.docx` yang valid
- Pastikan folder Input dan Output sudah ada
- Hasil dekripsi mungkin sedikit berbeda dari teks asli karena formatting Playfair (normal)